In [1]:
import os
import sys
sys.path.insert(0,'..')
from citylearn import  CityLearn
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from agents.DDPG.ddpg import DDPG

In [2]:
climate_zone = 5
sim_period = (0, 8760*4-1)
params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)),
        'building_attributes':'building_attributes.json',
        'weather_file':'weather_data.csv',
        'solar_profile':'solar_generation_1kW.csv',
        'carbon_intensity':'carbon_intensity.csv',
        'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
        'buildings_states_actions':'buildings_state_action_space.json',
        'simulation_period': sim_period,
        'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'],
        'central_agent': False,
        'save_memory': False }

env = CityLearn(**params)

In [3]:
observations_spaces, actions_spaces = env.get_state_action_spaces()
building_info = env.get_building_information()

In [5]:
params_agent = {'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
                 'buildings_states_actions':'buildings_state_action_space.json',
                 'building_info':building_info,
                 'observation_spaces':observations_spaces,
                 'action_spaces':actions_spaces}

# Instantiating the control agent(s)
agents = DDPG(**params_agent)

state = env.reset()
done = False

action = agents.select_action(state,0.2)
episode = 10
for i in range(episode):
    for i in range(1,2*8760):
        next_state, reward, done, _ = env.step(action)
        action_next = agents.select_action(next_state,0.2)
        agents.add_to_buffer(state, action, reward, next_state, done)
        state = next_state
        action = action_next

env.cost()

cpu


/Users/xiejiahan/PycharmProjects/RLinDR/CityLearn/agents/DDPG/ddpg.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(actions)


({'ramping': 0.0,
  '1-load_factor': 0.0,
  'average_daily_peak': 0.8051265,
  'peak_demand': 0.3937369,
  'net_electricity_consumption': 3.2713953e-05,
  'carbon_emissions': 3.3193413e-05,
  'total': 0.19982154907908503,
  'coordination_score': 0.2997158467769623},
 {'ramping_last_yr': 0.0,
  '1-load_factor_last_yr': 0.0,
  'average_daily_peak_last_yr': 0.8157906,
  'peak_demand_last_yr': 0.4450824,
  'net_electricity_consumption_last_yr': 0.00013231984,
  'carbon_emissions_last_yr': 0.00013720339,
  'coordination_score_last_yr': 0.3152182474732399,
  'total_last_yr': 0.22519439437019173})

In [ ]:
sim_period = (0, 8760*4 - 1)
interval = range(sim_period[0], sim_period[1])
plt.figure(figsize=(16,5))
plt.plot(env.net_electric_consumption_no_pv_no_storage[interval])
plt.plot(env.net_electric_consumption_no_storage[interval])
plt.plot(env.net_electric_consumption[interval], '--')
plt.xlabel('time (hours)')
plt.ylabel('kW')
plt.legend(['Electricity demand without storage or generation (kW)',
            'Electricity demand with PV generation and without storage(kW)',
            'Electricity demand with PV generation and using SAC for storage control (kW)'])

In [ ]:
# Plotting summer operation in the last year
interval = range(8760*3 + 24*30*6, 8760*3 + 24*30*6 + 24*10)
plt.figure(figsize=(16,5))
plt.plot(env.net_electric_consumption_no_pv_no_storage[interval])
plt.plot(env.net_electric_consumption_no_storage[interval])
plt.plot(env.net_electric_consumption[interval], '--')
plt.xlabel('time (hours)')
plt.ylabel('kW')
plt.legend(['Electricity demand without storage or generation (kW)',
            'Electricity demand with PV generation and without storage(kW)',
            'Electricity demand with PV generation and using RBC for storage(kW)'])

In [ ]:
building_number = 'Building_1'
interval = (range(24*30*6 + 8760*3,24*30*6 + 8760*3 + 24*4))
plt.figure(figsize=(12,8))
plt.plot(env.buildings[building_number].cooling_demand_building[interval])
plt.plot(env.buildings[building_number].cooling_storage_to_building[interval] - env.buildings[building_number].cooling_device_to_storage[interval])
plt.plot(env.buildings[building_number].cooling_device.cooling_supply[interval])
plt.plot(env.electric_consumption_cooling[interval])
plt.plot(env.buildings[building_number].cooling_device.cop_cooling[interval]*100,'--')
plt.plot(env.buildings[building_number].cooling_storage.soc[interval],'--')
plt.xlabel('time (hours)')
plt.ylabel('kW')
plt.legend(['Cooling Demand (kWh)',
            'Energy Balance of Chilled Water Tank (kWh)',
            'Heat Pump Total Cooling Supply (kWh)',
            'Heat Pump Electricity Consumption (kWh)',
            'Heat Pump COP x100',
            'Cooling Storage State of Charge (kWh)'])

In [ ]:
building_number = 'Building_9'
interval = range(8760*3 + 24*30*6, 8760*3 + 24*30*6 + 24*4)
plt.figure(figsize=(12,8))
plt.plot(env.buildings[building_number].cooling_storage_soc[interval])
plt.plot(env.buildings[building_number].dhw_storage_soc[interval])
plt.plot(env.buildings[building_number].electrical_storage_soc[interval])
plt.xlabel('time (hours)')
plt.ylabel('kW')
plt.legend(['Cooling Storage Device SoC',
            'Heating Storage Device SoC',
            'Electrical Storage Device SoC'])